## Deploy the Saved Model in the project to Deployment Space

### The following cell is a way to get the utility script required for this notebook. 
Since IBM CPD SaaS doesn't have a filesystem, this is the only reliable way to get scripts on the cloud environment. 
```
!rm -rf MLOps-CPD && git clone --quiet -b master https://github.com/IBM/MLOps-CPD.git
```
⚠️ Run the following cells only if you are executing on IBM CPD SaaS.

In [ ]:
#!rm -rf MLOps-CPD && git clone --quiet -b master https://github.com/IBM/MLOps-CPD.git

In [ ]:
#!mv MLOps-CPD MLOps_CPD

In [2]:
!pip install hyperopt ibm_aigov_facts_client ibm_watson_machine_learning ibm_watson_studio_pipelines lightgbm

  Obtaining dependency information for ibm_aigov_facts_client from https://files.pythonhosted.org/packages/a1/06/f38a9524334660c9c25b592ecd84c8a884641c95a9f324df107c2c9dabb9/ibm_aigov_facts_client-1.0.59-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 58.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.6/547.6 kB 58.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for gitpython<4,>=2.1.0 from https://files.pythonhosted.org/packages/67/50/742c2fb60989b76ccf7302c7b1d9e26505d7054c24f08cc7ec187faaaea7/GitPython-3.1.32-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements

In [3]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_machine_learning import APIClient
from ibm_aigov_facts_client import AIGovFactsClient
from ibm_watson_studio_pipelines import WSPipelines
from botocore.client import Config
import ibm_boto3
import pandas as pd
import json
import os
import requests
import pickle

from os.path import exists
if exists("utils/fs_utils.py") and exists("utils/catalog_utils.py"):
    from utils import fs_utils,catalog_utils
else:
    # If utils/fs_utils.py and utils/catalog_utils.py do not exist, we assume that you are running on CPDaaS
    # and will therefore import scripts from the freshly cloned repository
    from MLOps_CPD.utils import fs_utils, catalog_utils

## Succeeding cell contains the credentials for MLOps COS
```
## PROJECT COS 
AUTH_ENDPOINT = "https://iam.cloud.ibm.com/oidc/token"
ENDPOINT_URL = "https://s3.private.us.cloud-object-storage.appdomain.cloud"
API_KEY_COS = "xxx"
BUCKET_PROJECT_COS = "mlops-donotdelete-pr-qxxcecxi1dtw94"

## MLOPS COS
ENDPOINT_URL_MLOPS = "https://s3.jp-tok.cloud-object-storage.appdomain.cloud"
API_KEY_MLOPS = "xxx"
CRN_MLOPS = "xxx"
BUCKET_MLOPS  = "mlops-asset"

## CATALOG
CATALOG_NAME = "MLOps-ns"
```

## Pipeline Params

In [14]:
test_data_filename = "test_gcr.csv"
path = "mlops-dir"

In [15]:
## REMOTE AUTH

TOKEN = os.getenv("USER_ACCESS_TOKEN")


## LOCAL FS

path = os.getenv("path")
test_data_filename = os.getenv("test_data_filename")

model_path = os.getenv("model_path")

## WML / Factsheets

SPACE_ID = os.getenv("SPACE_ID")
MODEL_NAME = os.getenv("model_name")
DEPLOYMENT_NAME = os.getenv("deployment_name")

## Instantiate WML Client

In [21]:
WML_CREDENTIALS = {
   "token": TOKEN,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.6"
}

wml_client = APIClient(WML_CREDENTIALS)

'1.0.286'

In [28]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

### Helper Methods

In [29]:
def check_for_file_in_filesystem(path):
    """
    Check existence of path in filesystem
    """
    if os.path.exists(path):
        return True
    else:
        print("File not found in specified path.")
        return False     


def load_data_from_filesystem(path):
    """
    Check existence of path in filesystem.
    If it does exist, loads csv via path
    If it does NOT exist, try to load data from Db2
    """
    body = check_for_file_in_filesystem(path)
    if body:
        suffix = path[-3:]
        # Check whether path ends on csv
        if suffix == "csv":
            gcf_df = pd.read_csv(path)
        else:
            with open(path) as f:
                gcf_df = pickle.load(f)
        return gcf_df

# Deserialize model

TODO: Later get model from model inventory instead

In [30]:
with open(model_path, 'rb') as f:
    model = pickle.load(f)

### Load Sample Data 

In [31]:
payload_data = load_data_from_filesystem(test_data_path)
payload_data = payload_data.drop('Risk',axis=1)
fields = payload_data.columns.tolist()
values = [payload_data.values.tolist()[0]]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
json.dumps(payload_scoring)

'{"input_data": [{"fields": ["LoanDuration", "LoanAmount", "InstallmentPercent", "CurrentResidenceDuration", "Age", "ExistingCreditsCount", "Dependents", "Telephone", "OthersOnLoan", "ExistingSavings", "Job", "LoanPurpose", "CheckingStatus", "CreditHistory", "OwnsProperty", "Housing", "EmploymentDuration", "Sex"], "values": [[31, 1889, 3, 3, 32, 1, 1, "none", "none", "100_to_500", "skilled", "other", "0_to_200", "credits_paid_to_date", "savings_insurance", "own", "less_1", "female"]]}]}'

In [32]:
# TODO: Move to notebook 3 and skip (de)serialization process
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.2-py3.10")
model_props_gbt = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.DESCRIPTION: MODEL_NAME,
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client.repository.ModelMetaNames.TYPE: "scikit-learn_1.1"
}

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props_gbt, training_data=fields,training_target=values)
print(published_model_details)

{'entity': {'hybrid_pipeline_software_specs': [], 'label_column': 'l0', 'schemas': {'input': [{'fields': [{'name': 'f1', 'type': 'str'}], 'id': '1', 'type': 'struct'}], 'output': []}, 'software_spec': {'id': 'b56101f1-309d-549b-a849-eaa63f77b2fb', 'name': 'runtime-22.2-py3.10'}, 'training_data_references': [{'id': '1', 'location': {}, 'schema': {'fields': [{'name': 'f1', 'type': 'str'}], 'id': '1', 'type': 'list'}, 'type': 'fs'}], 'type': 'scikit-learn_1.1'}, 'metadata': {'created_at': '2023-08-01T17:02:12.179Z', 'description': 'test-model-lgbm', 'id': 'dfcfe9c3-432a-4f51-9aa4-8f720e028ac9', 'modified_at': '2023-08-01T17:02:13.287Z', 'name': 'test-model-lgbm', 'owner': '1000331062', 'resource_key': '797402f8-1aed-4463-a51c-14e0f0934bd6', 'space_id': '47e8a623-3312-4ccf-96c8-799a55c2b711'}, 'system': {'warnings': []}}


In [33]:
model_id = wml_client.repository.get_model_id(published_model_details)
model_id

'dfcfe9c3-432a-4f51-9aa4-8f720e028ac9'

## Promote the Model to deployment space and Deploy the Model

In [34]:
meta_data = {
    wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
    wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name": "S",
        "num_nodes": 1,
    }
}

deployment_details = wml_client.deployments.create(model_id, meta_props=meta_data)



#######################################################################################

Synchronous deployment creation for uid: 'dfcfe9c3-432a-4f51-9aa4-8f720e028ac9' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='90893f6b-e625-4abd-8c49-b94423f418d3'
------------------------------------------------------------------------------------------------




In [35]:
deployment_uid = wml_client.deployments.get_id(deployment_details)
deployment_uid

'90893f6b-e625-4abd-8c49-b94423f418d3'

## Score the Endpoint

### Model Testing on the Serving Endpoint



In [36]:
predictions = wml_client.deployments.score(deployment_uid, payload_scoring)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[0, [0.8620280281906896, 0.1379719718093104]]]}]}

### Test for Downstream Apps without using WML SDK.

In [37]:
# Rework test for local WML
#deploy_done = test_deployment(CLOUD_API_KEY, deployment_uid)
#deploy_done
deploy_done = True

## Save Params in WS Pipeline

In [40]:
deployment_done = {}
deployment_done['deployment_status'] = deploy_done
deployment_done['deployment_id'] = deployment_uid
deployment_done['model_id'] = model_id
deployment_done['space_id'] = SPACE_ID

In [41]:
pipelines_client = WSPipelines.from_token(TOKEN)
pipelines_client.store_results(deployment_done)

Running outside of Watson Studio Pipeline - storing results in the local filesystem for testing purposes...

  output paths:
    - "deployment_status": .ibm_watson_studio_pipelines/results/deployment_status
    - "deployment_id": .ibm_watson_studio_pipelines/results/deployment_id
    - "model_id": .ibm_watson_studio_pipelines/results/model_id
    - "space_id": .ibm_watson_studio_pipelines/results/space_id
